In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from simba.config import Config
from simba.simba.simba import Simba
from simba.simba.preprocessing_simba import PreprocessingSimba
from simba.simba.plotting import Plotting
from simba.simba.analog_discovery import AnalogDiscovery
import numpy as np
import spectrum_utils.plot as sup
from rdkit import Chem


Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality


* In this notebook, we load spectra from a mgf file and this is run on SIMBA to produce the corresponding edit distance and mces distances

In [3]:
config=Config()

In [4]:
model_location= '/Users/sebas/projects/data/best_model_generated_data_20250130.ckpt'

mgf_file = '/Users/sebas/projects/data/ALL_GNPS_NO_PROPOGATED_wb.mgf'
casmi_file= '/Users/sebas/projects/data/processed_massformer/spec_df.pkl' 

## Initialize Simba

In [5]:
simba= Simba(model_location, config=config, device='cpu', cache_embeddings=True)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/sebas/miniconda3/envs/transformers/lib/python3.11/site-packages/lightning/pytorch/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


## load mgf file

In [6]:
all_spectrums=PreprocessingSimba.load_spectra(casmi_file, config, use_gnps_format=True)

/Users/sebas/projects/data/processed_massformer/spec_df.pkl


In [7]:
len(all_spectrums)

158

In [8]:
np.random.seed(42)
random_indexes_0= np.random.randint(0,len(all_spectrums),30)
random_indexes_1= np.random.randint(0,len(all_spectrums),10)

In [9]:
## spectrums of the first index
import copy
all_spectrums_0 = copy.deepcopy([all_spectrums[index] for index in random_indexes_0])
all_spectrums_1 = copy.deepcopy([all_spectrums[index] for index in random_indexes_1])

In [10]:
len(all_spectrums_0)

30

## Predictions

In [11]:
sim_ed, sim_mces= simba.predict(all_spectrums_0, all_spectrums_1)

Preprocessing all the data ...


30it [00:00, 27395.85it/s]


Finished preprocessing 
loading data
Normalizing intensities
Preprocessing all the data ...


10it [00:00, 42538.58it/s]
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Finished preprocessing 
loading data
Normalizing intensities
running
Processing embeddings ...


...

/Users/sebas/projects/metabolomics/simba/transformers/spectrum_transformer_encoder_custom.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mass_precursor = torch.tensor(kwargs["precursor_mass"].float())
/Users/sebas/projects/metabolomics/simba/transformers/spectrum_transformer_encoder_custom.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  charge_precursor = torch.tensor(kwargs["precursor_charge"].float())
/Users/sebas/miniconda3/envs/transformers/lib/python3.11/site-packages/torch/nn/modules/transformer.py:408: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytor

running
Processing embeddings ...


Predicting: |                                                                                                 …

30it [00:00, 5586.45it/s]

Elapsed time: 0.10 seconds


## Ground truth

In [12]:
from simba.simba.ground_truth import GroundTruth

In [ ]:
ground_truth_ed=GroundTruth.compute_edit_distance(all_spectrums_0, all_spectrums_1)

In [ ]:
ground_truth_mces=GroundTruth.compute_mces(all_spectrums_0, all_spectrums_1)

## Comparison between predictions and ground truth

In [ ]:
from scipy.stats import spearmanr

In [ ]:
#!pip install myopic-mces==1.0.0

In [ ]:
Plotting.plot_mces(np.reshape(ground_truth_mces,-1), np.reshape(sim_mces,-1))

In [ ]:
Plotting.plot_cm(5-np.reshape(ground_truth_ed,-1), 5-np.reshape(sim_ed,-1))

## Check the closest spectra

* It returns a ranking matrix where if the score is higher, the similarity is higher based on reranking of MCES and Edit distance 

In [ ]:

ranking= AnalogDiscovery.compute_ranking(sim_mces, sim_ed)

In [ ]:
ground_truth_ed

## Plot the closest spectra

In [ ]:
index=5

In [ ]:
target_coordinates=np.argwhere(ground_truth_ed==5)

In [ ]:
target_index=target_coordinates[index][0]

In [ ]:
#best_match_index = np.argmax(ranking[target_index])
best_match_index =target_coordinates[index][1]

* Ground truth

In [ ]:
print(f'Edit distance {ground_truth_ed[target_index, best_match_index]}, MCES distance {ground_truth_mces[target_index, best_match_index]}')

* Prediction

In [ ]:
print(f'Edit distance {sim_ed[target_index, best_match_index]}, MCES distance {sim_mces[target_index, best_match_index]}')

In [ ]:
spectra_0= all_spectrums_0[target_index]

In [ ]:
spectra_1= all_spectrums_1[best_match_index]

In [ ]:
spectra_0.params['smiles']

In [ ]:
spectra_1.params['smiles']

In [ ]:
Chem.MolFromSmiles(spectra_0.params['smiles'])

In [ ]:
Chem.MolFromSmiles(spectra_1.params['smiles'])

In [ ]:
spectra_0.params

In [ ]:
spectra_1.params

In [ ]:
sup.mirror(spectra_0, spectra_1)